In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText, AutoTokenizer, pipeline, Trainer

processor = AutoProcessor.from_pretrained("bczhou/tiny-llava-v1-hf")
model = AutoModelForImageTextToText.from_pretrained("bczhou/tiny-llava-v1-hf")
tokenizer = AutoTokenizer.from_pretrained("bczhou/tiny-llava-v1-hf")

trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import pipeline
from PIL import Image
model_id = "bczhou/tiny-llava-v1-hf"
pipe = pipeline("image-to-text", model=model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [9]:
image = Image.open("./model/images/test.png")
prompt = "USER: <image>\nWhat is on this image\nASSISTANT:"
outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
print(outputs[0])


{'generated_text': "USER:  \nWhat is on this image\nASSISTANT: The image is a screenshot of a website page displaying information about the game Borderlands 1. The page is in French and features a large image of a character, possibly a zombie, with a description of the game. The page also includes a list of the game's features, such as the number of players, the number of levels, and the number of weapons. The website is designed in a modern and visually appealing style, with a focus on the game's content and features."}
